# Object changes

Analyse two snapshots for the aggregates that changed.

In [1]:
import json
import ipaddress
import pathlib

import pandas as pd

In [2]:
chunks = []

base_dir = pathlib.Path("~/Downloads").expanduser()

for path in base_dir.glob("*/output/rpki-client.json"):
    with path.open("r") as f:
        chunk = pd.DataFrame(json.load(f)["roas"])
        chunk['file'] = path.relative_to(base_dir)
        # And add address family
        chunk['af'] = chunk.prefix.map(lambda p: ipaddress.ip_network(p).version)
        # And prefix length (to chart later)
        chunk['prefix_length'] = chunk.prefix.map(lambda p: ipaddress.ip_network(p).prefixlen)

        chunks.append(chunk)
        
df = pd.concat(chunks)

In [3]:
df.file.unique()

array([PosixPath('rpki-20250326T200622Z/output/rpki-client.json'),
       PosixPath('rpki-20250326T233616Z/output/rpki-client.json')],
      dtype=object)

In [4]:
df_first = chunks[0].drop(columns="file")
df_second = chunks[1].drop(columns="file")

In [5]:
df_joined = df_first.set_index(["asn", "prefix", "maxLength"]).join(df_second.set_index(["asn", "prefix", "maxLength"]), lsuffix="_before", rsuffix="_after", how="left")
df_joined.sample(5)

,,,ta_before,expires_before,af_before,prefix_length_before,ta_after,expires_after,af_after,prefix_length_after
asn,prefix,maxLength,,,,,,,,
214790,5.101.47.0/24,24,ripe,1743080329,4,24,ripe,1.743088e+09,4.0,24.0
29119,78.136.107.0/24,24,ripe,1743080329,4,24,ripe,1.743098e+09,4.0,24.0
38491,121.127.4.0/24,24,apnic,1743608668,4,24,apnic,1.743609e+09,4.0,24.0
3209,2a02:908:2b20::/44,44,ripe,1743080329,6,44,ripe,1.743098e+09,6.0,44.0
1299,185.65.224.0/22,24,ripe,1743076886,4,22,ripe,1.743107e+09,4.0,22.0


In [6]:
df_joined[df_joined.prefix_length_after.isna() & df_joined.ta_before=="arin"]

,,,ta_before,expires_before,af_before,prefix_length_before,ta_after,expires_after,af_after,prefix_length_after
asn,prefix,maxLength,,,,,,,,


In [8]:
df_joined[(df_joined.ta_before=="apnic") & df_joined.prefix_length_after.isna()]

ta_before  expires_before  af_before  \
asn   prefix        maxLength                                        
18144 1.0.64.0/18   18            apnic      1743023427          4   
59125 1.21.96.0/20  24            apnic      1743023427          4   
      1.21.128.0/20 24            apnic      1743023427          4   
2514  1.33.0.0/16   24            apnic      1743022625          4   
                    16            apnic      1743022625          4   
...                                 ...             ...        ...   
37899 240b:252::/32 40            apnic      1743023427          6   
      240b:253::/32 40            apnic      1743023427          6   
2527  240d::/27     48            apnic      1743022968          6   
2516  240f::/24     24            apnic      1743022382          6   
      240f:100::/24 24            apnic      1743022382          6   

                               prefix_length_before ta_after  expires_after  \
asn   prefix        maxLength                                                 
18144 1.0.64.0/18   18                           18      NaN            NaN   
59125 1.21.96.0/20  24                           20      NaN            NaN   
      1.21.128.0/20 24                           20      NaN            NaN   
2514  1.33.0.0/16   24                           16      NaN            NaN   
                    16                           16      NaN            NaN   
...                                             ...      ...            ...   
37899 240b:252::/32 40                           32      NaN            NaN   
      240b:253::/32 40                           32      NaN            NaN   
2527  240d::/27     48                           27      NaN            NaN   
2516  240f::/24     24                           24      NaN            NaN   
      240f:100::/24 24                           24      NaN            NaN   

                               af_after  prefix_length_after  
asn   prefix        maxLength                                 
18144 1.0.64.0/18   18              NaN                  NaN  
59125 1.21.96.0/20  24              NaN                  NaN  
      1.21.128.0/20 24              NaN                  NaN  
2514  1.33.0.0/16   24              NaN                  NaN  
                    16              NaN                  NaN  
...                                 ...                  ...  
37899 240b:252::/32 40              NaN                  NaN  
      240b:253::/32 40              NaN                  NaN  
2527  240d::/27     48              NaN                  NaN  
2516  240f::/24     24              NaN                  NaN  
      240f:100::/24 24              NaN                  NaN  

[5131 rows x 8 columns]

In [9]:
df_joined[(df_joined.ta_before=="apnic") & df_joined.prefix_length_after.isna()].to_csv("20250326-apnic-dip.csv")

In [16]:
df_joined[(df_joined.ta_before=="apnic") & df_joined.prefix_length_after.isna()].reset_index()['asn'].unique().size

304